# NBA Sentiment Analysis

## Run Coreference resolution for pronouns 

In [ ]:
import datetime
import importlib
import pandas as pd

import analyze_scores
import data_manager as dm
import dir_config_manager
import criticism_nba
import entity_generator
import nba_schedule
import nba_data
import process_text
import pronoun_rename
import process_scores
import sentiment_analysis
import utils 
import you_pronoun 


dir_config = dir_config_manager.sample_dir_config

if 'sample_nba_data2' not in dir_config['data_dir']:
    dir_config = None
print(dir_config)

data_manager = dm.DataManager(dir_config=dir_config)


In [ ]:
for round_num in range(1, 5):
    pronoun_rename.split_and_save_data(data_manager, round_num, 
                                              is_clean_text=True, 
                                              max_num_comments=1000)

In [ ]:
round_num = 4
for round_num in range(1, 5):
    num_splits = data_manager.num_text_splits(round_num, data_type='preprocessed')
    for num_split in range(num_splits):
        df_comments, df_submi = data_manager.load_split_data(round_num, num_split)
        print(df_comments.shape, df_submi.shape)

        df_processed_text = pronoun_rename.run_pronoun_coref(
            df_comments, df_submi, data_manager, 200)
        data_manager.save_coreference_text(df_processed_text, round_num, num_split, 'processed')
        

In [ ]:
for round_num in range(1, 5):
    df_processed_text = data_manager.load_processed_text(round_num)
    print(df_processed_text.shape)

## Sentiment analysis

In [ ]:
data_manager = dm.DataManager(dir_config=dir_config)

In [ ]:
for round_num in range(1, 5):
    df_text = sentiment_analysis.create_sentiment_text(round_num, data_manager)
    print(df_text.shape)
    sentiment_analysis.split_and_save_sent_text_by_team(df_text, round_num, data_manager)


In [ ]:
data_manager = dm.DataManager(dir_config=dir_config)

In [ ]:
for round_num in range(1, 5):
    teams = utils.get_team_rounds(round_num)
    
    for team in teams:
        df_text = data_manager.load_sentiment(round_num, team)
        aspects = sentiment_analysis.get_aspects(team, data_manager)
        print(round_num, df_text.shape)

        df_all_scores = sentiment_analysis.score_aspect_sentiment(df_text, aspects)
        data_manager.save_sentiment(df_all_scores, round_num, team, data_type='sent_score')
        
    

In [ ]:
df_all_scores.shape

## Process and score 'You' pronoun

In [ ]:

data_manager = dm.DataManager(dir_config=dir_config)
you_pronoun.split_and_save_you_text(data_manager, max_num_comments=1000)



In [ ]:
you_pronoun.run_coref_pronoun_you(data_manager, interval=256)

In [ ]:
data_manager = dm.DataManager(dir_config=dir_config)

you_pronoun.get_sentiment_emma(data_manager)

## Plot Results

In [ ]:
data_manager = 0

In [ ]:
teams = [
        'Pelicans', 'Jazz', 'Nuggets', 'Timberwolves',
        'Hawks', 'Raptors', 'Bulls', 'Nets',
        'Suns', 'Grizzlies', '76ers', 'Bucks',
        'Mavericks', 'Heat', 'Warriors', 'Celtics']


plot_data_manager = dm.DataManager()

In [ ]:
df_all_comments = process_scores.get_all_comments(plot_data_manager, 4)

In [ ]:
sentiment_dict = process_scores.get_sentiment_scores(plot_data_manager, teams, df_all_comments)
df_sch = nba_schedule.getSchedule(2022)

In [ ]:
analyze_scores.plot_cmp_pre_and_post_eli(teams, sentiment_dict, df_sch)


In [ ]:
teams_opp_vs_non_opp = ['Suns', 'Grizzlies', '76ers', 'Bucks',
             'Mavericks', 'Heat',  'Celtics', 'Warriors']
analyze_scores.plot_opp_vs_non_opp(teams_opp_vs_non_opp, sentiment_dict, df_sch)

In [ ]:
analyze_scores.get_most_negative_sent_player(teams, sentiment_dict)

In [ ]:
# Get the least liked player.
df_scores = analyze_scores.get_most_least_liked(sentiment_dict, df_sch)
analyze_scores.plot_most_diff(df_scores)

In [ ]:
df_all_scores = process_scores.all_scores_get_dataframe(sentiment_dict)
analyze_scores.plt_round_three_line_plots(df_all_scores, df_sch)

In [ ]:
df_most_neg_team = analyze_scores.team_most_neg_toward(df_all_scores)

In [ ]:
df_resp_sent = plot_data_manager.load_sentiment('you', 'nba', 'sent_score')
df_sent_twd_user = criticism_nba.create_sent_twd_user(df_all_comments, df_resp_sent)



In [ ]:
sentiment_dict = process_scores.get_sentiment_scores(
    plot_data_manager, teams, df_all_comments)

In [ ]:
print(teams)

In [ ]:
df_plt = criticism_nba.sent_againt_user_after_criticism(teams, df_sent_twd_user, 
                                               df_resp_sent, df_all_comments)

In [ ]:
criticism_nba.plot_kde_sent_bef_vs_after_criticism(df_plt)
criticism_nba.plot_bar_sent_bef_vs_after_criticism(df_plt)

In [ ]:
df_plt_all = criticism_nba.sent_againt_team_after_criticism(df_sent_twd_user, df_resp_sent,
                                              sentiment_dict, df_all_comments)


In [ ]:
criticism_nba.plot_sent_againt_team_after_criticism(df_plt_all)


In [ ]:
criticism_nba.run_mixed_linear_model_regre(df_plt_all)
